In [26]:
import numpy as np
import time
from itertools import product
from math import sqrt
import random
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go


np.seterr(divide='ignore', invalid='ignore')
np.set_printoptions(precision=10)

In [27]:
def find_eig_by_power_method(
        matrix,
        *,
        eps,
        init_x=None,
        limit=10000,
):
    curr_x = np.ones(matrix.shape[0]) if init_x is None else init_x
    curr_x = curr_x / linalg.norm(curr_x)

    for step in range(limit):
        next_x = matrix.dot(curr_x)
        value = sqrt(next_x.dot(next_x) / curr_x.dot(curr_x))

        if linalg.norm(next_x - value * curr_x) / linalg.norm(curr_x) <= eps:
            return value, next_x, step + 1

        curr_x = next_x / linalg.norm(next_x)

    return value, next_x, step + 1

In [28]:
def find_eig_by_scalar_method(
    matrix,
    *,
    eps,
    init_x=None,
    limit=10000,
):
    curr_x = np.ones(matrix.shape[0]) if init_x is None else init_x
    
    curr_x, curr_y = curr_x / linalg.norm(curr_x), np.copy(curr_x)
    next_x, next_y = matrix.dot(curr_x), matrix.T.dot(curr_y)

    curr_value = next_x.dot(next_y) / curr_x.dot(next_y)

    for step in range(limit):
        curr_x, curr_y = next_x / linalg.norm(next_x), next_y / linalg.norm(next_y)
        next_x, next_y = matrix.dot(curr_x), matrix.T.dot(curr_y)

        next_value = next_x.dot(next_y) / curr_x.dot(next_y)

        if linalg.norm(next_value - curr_value) <= eps:
            return next_value, next_x, step + 1

        curr_value = next_value

    return next_value, next_x, step + 1

In [29]:
def find_smallest_eig_by_power_method(        
    matrix,
    *,
    eps,
    init_x=None,
    limit=10000
):
    eig, vector, step_count = find_eig_by_power_method(-matrix, eps=eps, init_x=init_x, limit=limit)
    return -eig, vector, step_count

def find_smallest_eig_by_scalar_method(        
    matrix,
    *,
    eps,
    init_x=None,
    limit=10000
):
    eig, vector, step_count = find_eig_by_scalar_method(-matrix, eps=eps, init_x=init_x, limit=limit)
    return -eig, vector, step_count

In [9]:
def generate_matrix(element_factory, size):
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )


def get_hilbert_matrix(size):
    return generate_matrix(lambda row, column: 1 / (row + column - 1), size)

In [41]:
def run_benchmark(matrix, eps):
    values, vectors = linalg.eig(matrix)
    argmax = abs(values).argmax()

    true_value, true_vector = abs(values)[argmax], vectors[:, argmax]

    power_value, power_vector, power_steps = find_eig_by_power_method(matrix, eps=eps)
    scalar_value, scalar_vector, scalar_steps = find_eig_by_scalar_method(matrix, eps=eps)

    return pd.Series([
        power_steps,
        abs(power_value - true_value),
        linalg.norm(abs(power_vector / linalg.norm(power_vector)) - abs(true_vector)),
        scalar_steps,
        abs(scalar_value - true_value),
        linalg.norm(abs(scalar_vector / linalg.norm(scalar_vector)) - abs(true_vector)),
    ])

def run_benchmark_smallest_eig(matrix, eps):
    values, vectors = linalg.eig(matrix)
    argmin = abs(values).argmin()

    true_value, true_vector = abs(values)[argmin], vectors[:, argmin]

    power_value, power_vector, power_steps = find_smallest_eig_by_power_method(matrix, eps=eps)
    scalar_value, scalar_vector, scalar_steps = find_smallest_eig_by_scalar_method(matrix, eps=eps)

    return pd.Series([
        power_steps,
        abs(power_value - true_value),
        linalg.norm(abs(power_vector / linalg.norm(power_vector)) - abs(true_vector)),
        scalar_steps,
        abs(scalar_value - true_value),
        linalg.norm(abs(scalar_vector / linalg.norm(scalar_vector)) - abs(true_vector)),
    ])

In [42]:
def draw_graphics(data, n):
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.power_steps, name='Степенной метод')
    fig.add_scatter(x=data.eps, y=data.scalar_steps, name='Метод скалярных произведений')

    fig.update_xaxes(title='ε', type='log', autorange='reversed',tickformat='.0e')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title=f'Зависимость количества шагов от ε (n = {n})')

    fig.show()

# Диагональная

In [43]:
def get_diagonal_matrix(size):
    return np.diag(range(1, size + 1))

In [44]:
diagonal_data = pd.DataFrame(product(range(10, 101, 10), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonal_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonal_data.apply(
    func=lambda row: run_benchmark(get_diagonal_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonal_data

,n,eps,power_steps,power_value_error,power_vector_error,scalar_steps,scalar_value_error,scalar_vector_error
0,10,1.000000e-02,45.0,8.934114e-05,8.727823e-03,15.0,3.866650e-02,0.185042
1,10,1.000000e-03,67.0,8.664323e-07,8.595043e-04,26.0,3.757591e-03,0.058126
2,10,1.000000e-04,89.0,8.402438e-09,8.464150e-05,37.0,3.699594e-04,0.018247
3,10,1.000000e-05,111.0,8.148504e-11,8.335248e-06,48.0,3.643486e-05,0.005726
4,10,1.000000e-06,133.0,7.887024e-13,8.208310e-07,59.0,3.588044e-06,0.001797
...,...,...,...,...,...,...,...,...
135,100,1.000000e-11,2522.0,0.000000e+00,9.816593e-12,1066.0,4.896350e-10,0.000022
136,100,1.000000e-12,2751.0,0.000000e+00,9.826986e-13,1181.0,4.853007e-11,0.000007
137,100,1.000000e-13,2980.0,0.000000e+00,9.837390e-14,1286.0,5.883294e-12,0.000002
138,100,1.000000e-14,3209.0,0.000000e+00,9.847806e-15,1363.0,1.250555e-12,0.000001


In [ ]:
draw_graphics(diagonal_data, 10)
draw_graphics(diagonal_data, 50)
draw_graphics(diagonal_data, 100)

In [ ]:
diagonal_data_smallest = pd.DataFrame(product(range(10, 101, 10), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonal_data_smallest[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonal_data_smallest.apply(
    func=lambda row: run_benchmark_smallest_eig(get_diagonal_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonal_data_smallest

In [ ]:
draw_graphics(diagonal_data_smallest, 10)
draw_graphics(diagonal_data_smallest, 50)
draw_graphics(diagonal_data_smallest, 100)

# Диагонально доминирующая

In [50]:
def get_diagonally_dominant_matrix(size):
    return np.eye(size) * 4 + np.eye(size, k=1) * -1 + np.eye(size, k=-1) * -1

In [ ]:
diagonally_dominant_data = pd.DataFrame(product([10, 20], [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonally_dominant_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonally_dominant_data.apply(
    func=lambda row: run_benchmark(get_diagonally_dominant_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonally_dominant_data

In [ ]:
draw_graphics(diagonally_dominant_data, 10)
draw_graphics(diagonally_dominant_data, 20)

In [ ]:
diagonally_dominant_data_smallest = pd.DataFrame(product([10, 20], [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonally_dominant_data_smallest[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonally_dominant_data_smallest.apply(
    func=lambda row: run_benchmark_smallest_eig(get_diagonally_dominant_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonally_dominant_data_smallest

In [ ]:
draw_graphics(diagonally_dominant_data_smallest, 10)
draw_graphics(diagonally_dominant_data_smallest, 20)

Матрица Гильберта

In [ ]:
hilbert_data = pd.DataFrame(product(range(2, 29), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
hilbert_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = hilbert_data.apply(
    func=lambda row: run_benchmark(get_hilbert_matrix(int(row.n)), row.eps),
    axis=1,
)
hilbert_data

In [ ]:
draw_graphics(hilbert_data, 2)
draw_graphics(hilbert_data, 14)
draw_graphics(hilbert_data, 28)

In [ ]:
hilbert_data_smallest = pd.DataFrame(product(range(2, 29), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
hilbert_data_smallest[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = hilbert_data_smallest.apply(
    func=lambda row: run_benchmark(get_hilbert_matrix(int(row.n)), row.eps),
    axis=1,
)
hilbert_data_smallest

In [ ]:
draw_graphics(hilbert_data_smallest, 2)
draw_graphics(hilbert_data_smallest, 14)
draw_graphics(hilbert_data_smallest, 28)

Обратные итерации со сдвигом

In [83]:
class EigenSpecifier:
    def __init__(self, max_iteration=1e7):
        self._previous_vector = None
        self._current_vector = None
        self.iterations_count = 0
        self._max_iteration = max_iteration

    def _is_iteration_finish(self, eps):
        if self.iterations_count <= 1:
            return False

        value = sqrt(self._current_vector.dot(self._current_vector) / self._previous_vector.dot(self._previous_vector))
        return linalg.norm(self._current_vector - value * self._previous_vector) / linalg.norm(self._previous_vector) <= eps or self.iterations_count > self._max_iteration

    def specify(
        self, matrix, eps: float, eigenvalue_approximation, initial_vector=None
    ):
        matrix_size = matrix.shape[0]
        new_matrix = matrix - eigenvalue_approximation * np.eye(matrix_size) 

        if initial_vector is None:
            initial_vector = np.ones(matrix_size)

        self.iterations_count = 0
        self._current_vector = initial_vector.copy()

        while not self._is_iteration_finish(eps):
            norm = np.max(np.abs(self._current_vector))
            self._current_vector /= linalg.norm(self._current_vector)

            self._previous_vector = self._current_vector
            self._current_vector = np.linalg.solve(new_matrix, self._current_vector)
            self.iterations_count += 1

        return 1 / np.mean(self._current_vector / self._previous_vector) + eigenvalue_approximation, self._current_vector

In [84]:
def get_gershgorin_circles(matrix):
    matrix_size = matrix.shape[0]
    _result = []
    for i in range(matrix_size):
        radius = sum(abs(matrix[i, j]) for j in range(matrix_size) if j != i)
        _result.append((matrix[i, i] - radius, matrix[i, i] + radius))
    return _result

In [87]:
A = np.array([
    [15, 3, 5, 2], 
    [-5, -23, 2, -1], 
    [12, -9, 35, -1], 
    [9, 2, 6, 36]
])

print(f'Точные значения: {[-22.090929928620027940, 11.170008850837157393, 38.387070273036965682, 35.533850804745904865]}')
print(f'Круги Гершгорина: {sorted(get_gershgorin_circles(A), key=lambda x: x[0])}')
print()

specifier = EigenSpecifier()
print(f'Приближенное значение: {11}')
print(f'Уточненное значение: {specifier.specify(A, 1e-7, 11)[0]}')
print()

print(f'Приближенное значение: {5}')
print(f'Уточненное значение: {specifier.specify(A, 1e-7, 5)[0]}')
print()

print(f'Приближенное значение: {-100}')
print(f'Уточненное значение: {specifier.specify(A, 1e-7, -100)[0]}')
print()

print(f'Приближенное значение: {30}')
print(f'Уточненное значение: {specifier.specify(A, 1e-7, 30)[0]}')
print()

print(f'Приближенное значение: {37}')
print(f'Уточненное значение: {specifier.specify(A, 1e-7, 37)[0]}')
print()

Точные значения: [-22.090929928620028, 11.170008850837158, 38.38707027303697, 35.5338508047459]
Круги Гершгорина: [(-31, -15), (5, 25), (13, 57), (19, 53)]

Приближенное значение: 11
Уточненное значение: 11.170008850483287

Приближенное значение: 5
Уточненное значение: 11.17000321087884

Приближенное значение: -100
Уточненное значение: -22.09189500538922

Приближенное значение: 30
Уточненное значение: 35.533846294124146

Приближенное значение: 37
Уточненное значение: 38.38706999861869

